<a href="https://colab.research.google.com/github/tsakailab/prml/blob/master/ipynb/ex_Perceptrons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# パーセプトロンの動作について学ぶ（Behavior of perceptrons）
----

氏名：

学生番号：

----
[A Neural Network Playground](http://playground.tensorflow.org/)を用いて，パーセプトロンの振る舞い考察しましょう．
Playground詳しい説明は[ここにあります](https://cloud.google.com/blog/products/ai-machine-learning/understanding-neural-networks-with-tensorflow-playground)．

以下の課題に取り組む際の注意です．
- "Run/Pause"の左のボタン"Reset the network"を用いて反復試行すること．必ず同じモデルに収束するとは限りません．
- 左下の"[REGENERATE]"で再生成したデータも試すこと．特定のデータについてではなく，普遍的に言える事柄を見つけましょう．
- 十分収束するまで観察すること．学習中のモデルよりも必ず良いモデルに収束するのでしょうか．
- 右下の"Show test data"で表示されるデータ（訓練データ以外のデータ）に対しても識別できているかどうかで評価すること．

特に，訓練データが少なく，ノイズの影響も無視できない問題設定にすると，考察に有益です．  
Ratio of training to test data: 20%，Noise: 25 の設定で実験します．

(a) [モデル：単純パーセプトロン，　データセット："Gaussian"](https://playground.tensorflow.org/#activation=sigmoid&batchSize=10&dataset=gauss&regDataset=reg-plane&learningRate=0.03&regularizationRate=0.01&noise=30&networkShape=&seed=0.85984&showTestData=false&discretize=true&percTrainData=20&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false&batchSize_hide=false)

(b) [モデル：3層パーセプトロン，　データセット："Circle"](https://playground.tensorflow.org/#activation=sigmoid&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.3&regularizationRate=0.01&noise=25&networkShape=1&seed=0.28240&showTestData=false&discretize=true&percTrainData=20&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false&batchSize_hide=false)

(c) [モデル：単純パーセプトロン，データセット："Circle"](https://playground.tensorflow.org/#activation=sigmoid&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.3&regularizationRate=0.01&noise=25&networkShape=&seed=0.07326&showTestData=false&discretize=true&percTrainData=20&x=true&y=true&xTimesY=true&xSquared=true&ySquared=true&cosX=false&sinX=true&cosY=false&sinY=true&collectStats=false&problem=classification&initZero=false&hideText=false&batchSize_hide=false)

まず，(a)で，単純パーセプトロンが線形識別器であることを確認してから，課題に着手しましょう．

----
## 基本課題（必須）

    1. (b)では，中間層に何個以上のノード（neurons）が必要ですか．その理由を述べてください（結果論は不可です）．
      また，多数のノード（例えば8個）で学習を続けると，どのような問題が生じますか．

（ここに回答を書いてください）



    2. (c)のモデルは単純パーセプトロンなのに，決定境界が直線でないのはなぜですか．
      また，入力に使われているどの特徴が強く使われる傾向がありますか．

（ここに回答を書いてください）



    3. 正則化（regularization）を"L1"に設定すると，モデルに特徴を選択させる学習の効果があります．この効果を(c)の例で具体的に説明してください．

（ここに回答を書いてください）



    4.その他，気づいたこと，調べたことがあれば書いてください．

（ここに回答を書いてください）




----
## 発展課題（任意）

Playgroundの右上に表示される "Test loss"，"Training loss" について考えましょう．


In [ ]:
#@title 例題用のデータを生成します．
import numpy as np
npos = 30
nneg = 30
np.random.seed(321)
X = np.r_[np.random.randn(npos, 2) + [3, 3], np.random.randn(nneg, 2)]
# [1,1,...,1,-1,-1,...,-1]
y = np.array([1] * npos + [-1] * nneg)

In [ ]:
#@title グラフを描く関数を定義します．
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import TwoSlopeNorm as tsn

def plot2d_classification(decision_function, X_train, y_train, X_test=None, y_test=None, dx=0.02, cmap=plt.cm.bwr, xlim=None, ylim=None, levels=None, colors='k', bins=None):

    fig, axes = plt.subplots(figsize=(12,8), nrows=1, ncols=2)

    ax = axes[0]
    if xlim is None:
        xlim = [X_train[:, 0].min() - .5, X_train[:, 0].max() + .5]
    if ylim is None:
        ylim = [X_train[:, 1].min() - .5, X_train[:, 1].max() + .5]

    xx, yy = np.meshgrid(np.arange(xlim[0], xlim[1], dx), np.arange(ylim[0], ylim[1], dx))    

    # Show prediction (P(y=+1 | X) by color by assigning a color to each point in the mesh [x_min, x_max]x[y_min, y_max].
    Z = decision_function(np.c_[xx.ravel(), yy.ravel()])
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    norm = tsn(vmin=np.minimum(Z[:].min(),-1e-6), vcenter=0, vmax=np.maximum(Z[:].max(),1e-6))
    ax.pcolor(xx, yy, Z, cmap=cmap, alpha=0.1, edgecolors=None, norm=norm)
    if levels is not None:
        lvls = list(levels.keys())
        linestyles = list(levels.values())
        ax.contour(xx, yy, Z, levels=lvls, colors=colors, linestyles=linestyles, alpha=0.5)

    # Plot the training points
    ax.scatter(X_train[y_train>0, 0], X_train[y_train>0, 1], c='r',  marker='s', edgecolors='k', label='Training data', alpha=1)
    ax.scatter(X_train[y_train<=0, 0], X_train[y_train<=0, 1], c='b', marker='o', edgecolors='k', label='Training data', alpha=1)
        
    # and testing points if given
    if X_test is not None and y_test is not None:
        ax.scatter(X_test[y_test>0, 0], X_test[y_test>0, 1], c='k',  marker='s', edgecolors='k', label='Test data', alpha=1)
        ax.scatter(X_test[y_test<=0, 0], X_test[y_test<=0, 1], c='k', marker='o', edgecolors='k', label='Test data', alpha=1)
        plt.legend(loc="upper right", fontsize=16, frameon=True)
        ax.get_legend().legendHandles[2].set_color('k')
        ax.get_legend().legendHandles[3].set_color('k')

    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    plt.axis('tight')
    plt.xlabel('x1', fontsize=16)
    plt.ylabel('x2', fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    ax.set_aspect('equal')
    #plt.gca().set_aspect('equal')
    plt.tight_layout()


    if bins is None:
        bins = len(y_train) // 4

    ax = axes[1]
    pred = decision_function(X_train)
    plt.hist( [ pred[y_train>0], pred[y_train<=0] ], bins=bins, histtype='stepfilled', density=False, alpha=0.5, color=['r', 'b'], label=['$y=+1$', '$y=-1$'])
    if X_test is not None and y_test is not None:
        pred = decision_function(X_test)
        plt.hist( [ pred[y_test>0], pred[y_test<=0] ], bins=bins, histtype='stepfilled', density=False, alpha=0.3, color=['r', 'b'], label=['$y_{test}=+1$', '$y_{test}=-1$'])
    plt.xlabel("$g(x)$", fontsize=16)
    plt.ylabel("Frequency", fontsize=16)
    plt.legend(loc="upper right", fontsize=16, frameon=True)
    plt.axis('tight')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    from matplotlib.ticker import FormatStrFormatter
    #plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%1.0f'))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%1.0f'))
    ax.set_aspect(0.8)
    plt.tight_layout()

### 損失関数を定義します．

In [ ]:
import numpy as np

def LogisticLoss(y, g):
    return np.mean(np.log(1 + np.exp(-y*g)))

def HingeLoss(y, g):
    return np.mean(np.maximum(1.0 - y*g, 0))

# 線形識別関数
def g(X, w):
    return w[0] + np.dot(X, w[1:])

### $[w_0, w_1, w_2]^\top$はパーセプトロンの重みです．表示される損失関数の値がなるべく小さくなるように調整してください．
    1. どのようなとき，損失が大きく・小さくなりますか．

（ここに回答を書いてください）


    2. w0, w1, w2 によって決定境界はどのように変わりますか．

（ここに回答を書いてください）




In [ ]:
from sklearn.linear_model import Hinge
#@title {run: "auto"}
w0 = 0.1 #@param{type:"slider", min:-5, max:5, step:0.1}
w1 = -1 #@param{type:"slider", min:-5, max:5, step:0.1}
w2 = -3 #@param{type:"slider", min:-5, max:5, step:0.1}
w = np.array( [w0, w1, w2] )

ll = LogisticLoss(y, g(X, w))
hl = HingeLoss(y, g(X, w))
print("Logistic loss = %f,  Hinge loss =%f" % (ll, hl))
plot2d_classification(lambda X: g(X, w), X, y, levels={0.0:'-'}, dx=0.2)
#histogram_predict(lambda X: g(X, w), X, y)